In [2]:
# Flux at spacecraft:
import numpy as np
import pickle
from build_database import flux_obj
from scipy import interpolate
from matplotlib import pyplot as plt
from GLD_file_tools import GLD_file_tools
from satellite import Satellite
import datetime
import ephem
from coordinate_structure import coordinate_structure
from coordinate_structure import transform_coords
from longitude_scaling import longitude_scaling
from ionoAbsorp import ionoAbsorp
import os
from mpl_toolkits.basemap import Basemap
from precip_model import precip_model
import itertools
from matplotlib.widgets import Slider
%matplotlib inline

# --------------- Latex Plot Beautification --------------------------
fig_width_pt = 650.0  # Get this from LaTeX using \showthe\columnwidth
inches_per_pt = 1.0/72.27               # Convert pt to inch
golden_mean = (np.sqrt(5)-1.0)/2.0         # Aesthetic ratio
fig_width = fig_width_pt*inches_per_pt  # width in inches
fig_height = fig_width*golden_mean      # height in inches
fig_size =  [fig_width+1,fig_height+1]
params = {'backend': 'ps',
          'axes.labelsize': 14,
          'text.fontsize': 14,
          'legend.fontsize': 10,
          'xtick.labelsize': 10,
          'ytick.labelsize': 10,
          'text.usetex': False,
          'figure.figsize': fig_size}
plt.rcParams.update(params)
# --------------- Latex Plot Beautification --------------------------



In [ ]:
GLD_root  = 'alex/array/home/Vaisala/feed_data/GLD'
NLDN_root = 'alex/array/home/Vaisala/feed_data/NLDN'

sat_TLE  = ["1 40378U 15003C   15293.75287141  .00010129  00000-0  48835-3 0  9990",
            "2 40378  99.1043 350.5299 0153633 201.4233 158.0516 15.09095095 39471"]

# Satellite object
sat = Satellite(sat_TLE[0], sat_TLE[1],'Firebird 4')
# Lightning-gettin' object
gld = GLD_file_tools(GLD_root, prefix='GLD')

# Column ind
lat_ind = 7;
lon_ind = 8;
mag_ind = 9;

In [42]:
class measurement_model(object):
    '''Instantiate this bad boy to make precipitation measurements'''
    
    def __init__(self,
                 database='database.pkl',
                 GLD_root = 'alex/array/home/Vaisala/feed_data/GLD'):
    
        self.m = precip_model(database)
        
        self.RES_DT = self.m.db[self.m.db.keys()[0]].RES_DT
        self.RES_FINT = self.m.db[self.m.db.keys()[0]].RES_FINT
        
        
        self.td = datetime.timedelta(seconds = 60) # Maximum previous time to examine flashes in.
        
        # Lightning database
        self.gld = GLD_file_tools(GLD_root, prefix='GLD')
        # Column indices
        self.lat_ind = 7;
        self.lon_ind = 8;
        self.mag_ind = 9;
        
    def get_measurement(self, in_time, coordinates, mode='continuous'):
        # Get flashes within timeframe:
        flashes, flash_times = self.gld.load_flashes(in_time, self.td)
        flashes = flashes[:,(self.lat_ind, self.lon_ind, self.mag_ind, self.mag_ind)]
        flash_coords = transform_coords(flashes[:,0], flashes[:,1], np.zeros_like(flashes[:,0]), 'geographic', 'geomagnetic')
        flashes[:,:2] = flash_coords[:,:2]
        flashes[:,3] = [(in_time - s).microseconds*1e-6 + (in_time - s).seconds for s in flash_times]

        
        # So! No we have an array of relevant flashes -- lat, lon, mag, time offset.
        # Let's model the flux at the satellite.
        flux = 0
        
        time_sampling_vector = np.linspace(-self.RES_FINT,0,np.round(self.RES_FINT/self.RES_DT))
        for f in flashes:
            #print td.seconds - f[3]   
            temp = ( self.m.get_precip_at(f[0], coordinates.lat(), time_sampling_vector + f[3]) *
                      self.m.get_longitude_scaling(f[0], f[1], coordinates.lon(), I0=f[2]) * self.RES_DT )
            flux += np.sum(temp)
            #print temp
        
        return flux

In [43]:
GLD_root  = 'alex/array/home/Vaisala/feed_data/GLD'
NLDN_root = 'alex/array/home/Vaisala/feed_data/NLDN'

sat_TLE  = ["1 40378U 15003C   15293.75287141  .00010129  00000-0  48835-3 0  9990",
            "2 40378  99.1043 350.5299 0153633 201.4233 158.0516 15.09095095 39471"]

# Satellite object:
sat = Satellite(sat_TLE[0], sat_TLE[1],'Firebird 4')

# Measurement object:
f = measurement_model(database = "database_counts.pkl")

# ---- Do The Thing:
inTime = "2015-11-01T00:45:00"
plottime = datetime.datetime.strptime(inTime,  "%Y-%m-%dT%H:%M:%S")

sat.compute(plottime)
sat.coords.transform_to('geomagnetic')

print f.get_measurement(plottime, sat.coords)



0.00553127648018


In [37]:
print f.RES_DT


0.05


In [ ]:
# Input time

inTime = "2015-11-01T00:45:00"
td = datetime.timedelta(seconds = 60) # Maximum time back to check for lightning (pulse can be up to a minute! But how patient are we, really)
plottime = datetime.datetime.strptime(inTime,  "%Y-%m-%dT%H:%M:%S")

print plottime

# Get satellite location
sat.compute(plottime)
sat.coords.transform_to('geomagnetic')

# Flux modeler:
m = precip_model("database_multiple.pkl")

# Get flashes within timeframe:
flashes, flash_times = gld.load_flashes(plottime, td)
flashes = flashes[:,(lat_ind, lon_ind, mag_ind, mag_ind)]
flash_coords = transform_coords(flashes[:,0], flashes[:,1], np.zeros_like(flashes[:,0]), 'geographic', 'geomagnetic')
flashes[:,:2] = flash_coords[:,:2]
flashes[:,3] = [(plottime - s).microseconds*1e-6 + (plottime - s).seconds for s in flash_times]

# scale_factors = np.array([m.get_longitude_scaling(f[0],f[1], sat.coords.lon(), db_scaling = True) for f in flashes])
# print np.max(scale_factors)
# plt.plot(scale_factors,marker='.')

# # Mask out insignificant flashes:
# mask = ( (np.abs(flashes[:,0]) > 10) &
#          (np.abs(flashes[:,0]) < 60) &
#          (scale_factors > -50)[:,0]
#        )

# masked_flashes = flashes[mask,:]
# tmp = transform_coords(masked_flashes[:,0],
#                        masked_flashes[:,1],
#                        np.zeros(np.sum(mask)),'geomagnetic', 'geographic')
# masked_flashes[:,0:2] = tmp[:,0:2]


# So! No we have an array of relevant flashes -- lat, lon, mag, time offset.
# Let's model the flux at the satellite.
flux = 0
time_sampling_vector = np.linspace(-10,0,10)
for f in flashes:
    #print td.seconds - f[3]   
    temp = ( m.get_precip_at(f[0], sat.coords.lat(), time_sampling_vector + f[3]) *
              m.get_longitude_scaling(f[0], f[1], sat.coords.lon(), I0=f[2]) )
    flux += np.sum(temp)
    #print temp
print np.log10(flux)






sat.coords.transform_to('geographic')

# ------ Basemap Plot -------
map = Basemap(projection='mill',lon_0=0)
map.drawcoastlines()
map.drawparallels(np.arange(-90,90,30),labels=[1,0,0,0])
map.drawmeridians(np.arange(map.lonmin,map.lonmax+30,60),labels=[0,0,0,1])
map.drawmapboundary(fill_color='b')
map.fillcontinents(color='white',lake_color='b')

x,y = map(masked_flashes[:,1], masked_flashes[:,0])
sx,sy = map(sat.coords.lon(),sat.coords.lat())
#mag = np.log2(abs(flashes[:,9]))
points = map.plot(x,y,'ro',markeredgecolor=[1,0,0], markersize=2)[0]
sats   = map.plot(sx,sy,'gx')[0]
print plottime
CS=map.nightshade(plottime,alpha=0.1)

plt.title('GLD flashes, %s (UTC)\nHold Time = %s sec' % (plottime, td.seconds))
plt.draw()
